In [2]:
pip install stockfish

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from stockfish import Stockfish
import pandas as pd
import chess
import chess.pgn
import io
import csv
stockfish = Stockfish(path="/Users/awasthi/Desktop/stockfish/stockfish-macos-m1-apple-silicon")

def get_evaluation(fen):
    try:
        stockfish.set_fen_position(fen)
        evaluation = stockfish.get_evaluation()
        if evaluation is None:
            raise ValueError("Stockfish returned None for the evaluation.")
        return evaluation
    except Exception as e:
        print(f"Error evaluating FEN {fen}: {e}")
        return None

def pgn_to_fens(pgn_text):
    """Convert a PGN game string to a list of FEN positions"""
    game = chess.pgn.read_game(io.StringIO(pgn_text))
    if game is None:
        return []
    
    board = game.board()
    fens = [board.fen()]
    
    for move in game.mainline_moves():
        board.push(move)
        fens.append(board.fen())
    
    return fens

def process_games_csv(input_file, output_file):
    try:
        df = pd.read_csv(input_file)
        if 'moves' not in df.columns:
            raise ValueError("CSV must contain a 'moves' column with PGN format games")
        
        with open(output_file, 'w', newline='') as f_out:
            writer = csv.writer(f_out)
            writer.writerow(['FEN', 'Evaluation'])
            
            for i, row in df.iterrows():
                pgn_text = row['moves']
                fens = pgn_to_fens(pgn_text)
                
                print(f"Processing game {i+1}/{len(df)}: {len(fens)} positions")
                
                for fen in fens:
                    evaluation = get_evaluation(fen)
                    if evaluation:
                        writer.writerow([fen, evaluation])
                    
        print(f"Completed! Results saved to {output_file}")
        
    except Exception as e:
        print(f"Error processing CSV: {e}")

input_csv = "chess_games.csv"
output_csv = "yooooo.csv"
    
process_games_csv(input_csv, output_csv)